# [The Boston Light Swim](http://bostonlightswim.org/)

Fetching near real time Sea Surface Temperature time-series data from all avialble source and compare them.

TODO: http://www.mdpi.com/2077-1312/4/4/68/html

In [1]:
import os
import sys
import time
import warnings

ioos_tools = os.path.join(os.path.pardir)
sys.path.append(ioos_tools)

# Suppresing warnings for a "pretty output."
# Remove this line to debug any possible issues.
warnings.simplefilter("ignore")

start_time = time.time()

### Configuration

In [2]:
%%writefile config.yaml

# Specify a YYYY-MM-DD hh:mm:ss date or integer day offset.
# If both start and stop are offsets they will be computed relative to datetime.today() at midnight.
# Use the dates commented below to reproduce the last Boston Light Swim event forecast.
date:
    start: -5 # 2016-8-16 00:00:00
    stop: +4 # 2016-8-29 00:00:00

run_name: 'latest'

# Boston harbor.
region:
    bbox: [-71.3, 42.03, -70.57, 42.63]
    crs: 'urn:ogc:def:crs:OGC:1.3:CRS84'

sos_name: 'sea_water_temperature'

cf_names:
    - sea_water_temperature
    - sea_surface_temperature
    - sea_water_potential_temperature
    - equivalent_potential_temperature
    - sea_water_conservative_temperature
    - pseudo_equivalent_potential_temperature

units: 'celsius'

catalogs:
    - http://www.ngdc.noaa.gov/geoportal/csw
    - https://dev-catalog.ioos.us/csw
    - http://gamone.whoi.edu/csw

# See https://raw.githubusercontent.com/OSGeo/Cat-Interop/master/LinkPropertyLookupTable.csv
service_strings:
    opendap:
        - OPeNDAP:OPeNDAP
        - urn:x-esri:specification:ServiceType:odp:url
    sos:
        - urn:x-esri:specification:ServiceType:sos:url
        - OGC:SOS

Overwriting config.yaml


In [3]:
from datetime import datetime
from ioos_tools.ioos import parse_config

config = parse_config('config.yaml')

save_dir = os.path.abspath(config['run_name'])


def _reload_log():
    """IPython workaround."""
    import imp
    import logging as log
    imp.reload(log)
    return log


def start_log(save_dir):
    import shutil
    log = _reload_log()
    if os.path.exists(save_dir):
        shutil.rmtree(save_dir)
    os.makedirs(save_dir)

    log.captureWarnings(True)
    LOG_FILENAME = 'log.txt'
    LOG_FILENAME = os.path.join(save_dir, LOG_FILENAME)
    formatter = '%(asctime)s %(levelname)s: %(message)s'
    log.basicConfig(filename=LOG_FILENAME,
                    filemode='w',
                    format=formatter,
                    datefmt='%I:%M:%S',
                    level=log.INFO)
    return log


log = start_log(save_dir)
fmt = '{:*^64}'.format
log.info(fmt('Saving data inside directory {}'.format(save_dir)))
log.info(fmt(' Run information '))
log.info('Run date: {:%Y-%m-%d %H:%M:%S}'.format(datetime.utcnow()))
log.info('Start: {:%Y-%m-%d %H:%M:%S}'.format(config['date']['start']))
log.info('Stop: {:%Y-%m-%d %H:%M:%S}'.format(config['date']['stop']))
log.info('Bounding box: {0:3.2f}, {1:3.2f},'
         '{2:3.2f}, {3:3.2f}'.format(*config['region']['bbox']))

### Create the data filter

In [4]:
def make_filter(config):
    from owslib import fes
    from ioos_tools.ioos import fes_date_filter
    kw = dict(wildCard='*', escapeChar='\\',
              singleChar='?', propertyname='apiso:AnyText')

    or_filt = fes.Or([fes.PropertyIsLike(literal=('*%s*' % val), **kw)
                      for val in config['cf_names']])

    # Exclude ROMS Averages and History files.
    not_filt = fes.Not([fes.PropertyIsLike(literal='*Averages*', **kw)])

    begin, end = fes_date_filter(config['date']['start'],
                                 config['date']['stop'])
    bbox_crs = fes.BBox(config['region']['bbox'],
                        crs=config['region']['crs'])
    return [fes.And([bbox_crs, begin, end, or_filt, not_filt])]


filter_list = make_filter(config)

In [5]:
from ioos_tools.ioos import service_urls
from owslib.csw import CatalogueServiceWeb

# Logging info.
fmt = '{:*^64}'.format
log.info(fmt(' Catalog information '))

# Check for the strings at
# https://raw.githubusercontent.com/OSGeo/Cat-Interop/master/LinkPropertyLookupTable.csv
opendap_strings = config['service_strings']['opendap']
sos_strings = config['service_strings']['sos']

dap_urls = []
sos_urls = []
for endpoint in config['catalogs']:
    log.info(fmt(' CSW '))
    log.info("URL: {}".format(endpoint))

    csw = CatalogueServiceWeb(endpoint, timeout=120)
    csw.getrecords2(constraints=filter_list, maxrecords=1000, esn='full')
    dap = service_urls(csw.records, services=opendap_strings)
    sos = service_urls(csw.records, services=sos_strings)
    dap_urls.extend(dap)
    sos_urls.extend(sos)

    log.info("CSW version: {}".format(csw.version))
    log.info("Number of datasets available: {}".format(len(csw.records.keys())))

    for rec, item in csw.records.items():
        log.info('{}'.format(item.title))
    if sos:
        log.info(fmt(' SOS '))
        for url in sos:
            log.info('{}'.format(url))
    if dap:
        log.info(fmt(' DAP '))
        for url in dap:
            log.info('{}.html'.format(url))

# Get only unique endpoints.
dap_urls = list(set(dap_urls))
sos_urls = list(set(sos_urls))

In [6]:
from ioos_tools.ioos import is_station

# Filter out some station endpoints.
non_stations = []
for url in dap_urls:
    try:
        if not is_station(url):
            non_stations.append(url)
    except (RuntimeError, OSError) as e:
        log.warn("Could not access URL {}. {!r}".format(url, e))

dap_urls = non_stations

log.info(fmt(' Filtered DAP '))
for url in dap_urls:
    log.info('{}.html'.format(url))

### NdbcSos

In [7]:
from pyoos.collectors.ndbc.ndbc_sos import NdbcSos

collector_ndbc = NdbcSos()

collector_ndbc.set_bbox(config['region']['bbox'])
collector_ndbc.end_time = config['date']['stop']
collector_ndbc.start_time = config['date']['start']
collector_ndbc.variables = [config['sos_name']]

ofrs = collector_ndbc.server.offerings
title = collector_ndbc.server.identification.title
log.info(fmt(' NDBC Collector offerings '))
log.info('{}: {} offerings'.format(title, len(ofrs)))

In [8]:
import pandas as pd
from ioos_tools.ioos import collector2table, to_html

ndbc = collector2table(collector=collector_ndbc,
                       config=config,
                       col='sea_water_temperature (C)')

if ndbc:
    data = dict(
        station_name=[s._metadata.get('station_name') for s in ndbc],
        station_code=[s._metadata.get('station_code') for s in ndbc],
        sensor=[s._metadata.get('sensor') for s in ndbc],
        lon=[s._metadata.get('lon') for s in ndbc],
        lat=[s._metadata.get('lat') for s in ndbc],
        depth=[s._metadata.get('depth') for s in ndbc],
    )

table = pd.DataFrame(data).set_index('station_code')
to_html(table)

,depth,lat,lon,sensor,station_name
station_code,,,,,
44013,0.6,42.35,-70.69,urn:ioos:sensor:wmo:44013::watertemp1,"BOSTON 16 NM East of Boston, MA"
44029,1.0,42.52,-70.57,urn:ioos:sensor:wmo:44029::ct1,Buoy A0102 - Mass. Bay/Stellwagen


### CoopsSoS

In [9]:
from pyoos.collectors.coops.coops_sos import CoopsSos

collector_coops = CoopsSos()

collector_coops.set_bbox(config['region']['bbox'])
collector_coops.end_time = config['date']['stop']
collector_coops.start_time = config['date']['start']
collector_coops.variables = [config['sos_name']]

ofrs = collector_coops.server.offerings
title = collector_coops.server.identification.title
log.info(fmt(' Collector offerings '))
log.info('{}: {} offerings'.format(title, len(ofrs)))

In [10]:
coops = collector2table(collector=collector_coops,
                        config=config,
                        col='sea_water_temperature (C)')

if coops:
    data = dict(
        station_name=[s._metadata.get('station_name') for s in coops],
        station_code=[s._metadata.get('station_code') for s in coops],
        sensor=[s._metadata.get('sensor') for s in coops],
        lon=[s._metadata.get('lon') for s in coops],
        lat=[s._metadata.get('lat') for s in coops],
        depth=[s._metadata.get('depth') for s in coops],
    )

table = pd.DataFrame(data).set_index('station_code')
to_html(table)

,depth,lat,lon,sensor,station_name
station_code,,,,,
8443970,None,42.3548,-71.0534,urn:ioos:sensor:NOAA.NOS.CO-OPS:8443970:E1,"Boston, MA"


### Join CoopsSoS and NdbcSos in uniform 1-hour time base series for model/data comparison

In [11]:
data = ndbc + coops

index = pd.date_range(start=config['date']['start'].replace(tzinfo=None),
                      end=config['date']['stop'].replace(tzinfo=None),
                      freq='1H')

# Preserve metadata with `reindex`.
observations = []
for series in data:
    _metadata = series._metadata
    obs = series.reindex(index=index, limit=1, method='nearest')
    obs._metadata = _metadata
    observations.append(obs)

### Save simpler station code/name file

In [12]:
import iris
from ioos_tools.tardis import series2cube

attr = dict(
    featureType='timeSeries',
    Conventions='CF-1.6',
    standard_name_vocabulary='CF-1.6',
    cdm_data_type="Station",
    comment="Data from http://opendap.co-ops.nos.noaa.gov"
)

log.info(fmt(' Observations '))
outfile = os.path.join(save_dir, 'OBS_DATA.nc')

cubes = iris.cube.CubeList(
    [series2cube(obs, attr=attr) for obs in observations]
)

iris.save(cubes, outfile)

### Loop discovered models and save the nearest time-series

In [13]:
from iris.exceptions import (CoordinateNotFoundError, ConstraintMismatchError,
                             MergeError)
from ioos_tools.ioos import get_model_name
from ioos_tools.tardis import quick_load_cubes, proc_cube, is_model, get_surface

log.info(fmt(' Models '))
cubes = dict()
for k, url in enumerate(dap_urls):
    log.info('\n[Reading url {}/{}]: {}'.format(k+1, len(dap_urls), url))
    try:
        cube = quick_load_cubes(url, config['cf_names'],
                                callback=None, strict=True)
        if is_model(cube):
            cube = proc_cube(cube,
                             bbox=config['region']['bbox'],
                             time=(config['date']['start'],
                                   config['date']['stop']),
                             units=config['units'])
        else:
            log.warning("[Not model data]: {}".format(url))
            continue
        cube = get_surface(cube)
        mod_name = get_model_name(url)
        cubes.update({mod_name: cube})
    except (RuntimeError, ValueError,
            ConstraintMismatchError, CoordinateNotFoundError,
            IndexError) as e:
        log.warning('Cannot get cube for: {}\n{}'.format(url, e))

In [14]:
import iris
from iris.pandas import as_series
from ioos_tools.tardis import (make_tree, get_nearest_water,
                               add_station, ensure_timeseries, remove_ssh)

for mod_name, cube in cubes.items():
    fname = '{}.nc'.format(mod_name)
    fname = os.path.join(save_dir, fname)
    log.info(fmt(' Downloading to file {} '.format(fname)))
    try:
        tree, lon, lat = make_tree(cube)
    except CoordinateNotFoundError as e:
        log.warning('Cannot make KDTree for: {}'.format(mod_name))
        continue
    # Get model series at observed locations.
    raw_series = dict()
    for obs in observations:
        obs = obs._metadata
        station = obs['station_code']
        try:
            kw = dict(k=10, max_dist=0.08, min_var=0.01)
            args = cube, tree, obs['lon'], obs['lat']
            try:
                series, dist, idx = get_nearest_water(*args, **kw)
            except RuntimeError as e:
                log.info('Cannot download {!r}.\n{}'.format(cube, e))
                series = None
        except ValueError as e:
            status = "No Data"
            log.info('[{}] {}'.format(status, obs['station_name']))
            continue
        if not series:
            status = "Land   "
        else:
            raw_series.update({station: series})
            series = as_series(series)
            status = "Water  "
        log.info('[{}] {}'.format(status, obs['station_name']))
    if raw_series:  # Save cube.
        for station, cube in raw_series.items():
            cube = add_station(cube, station)
            cube = remove_ssh(cube)
        try:
            cube = iris.cube.CubeList(raw_series.values()).merge_cube()
        except MergeError as e:
            log.warning(e)
        ensure_timeseries(cube)
        iris.save(cube, fname)
        del cube
    log.info('Finished processing [{}]'.format(mod_name))

### Load configuration

In [15]:
from ioos_tools.ioos import parse_config

config = parse_config('config.yaml')

save_dir = os.path.join(os.path.abspath(config['run_name']))

### Skill 1: Model Bias (or Mean Bias)

The bias skill compares the model mean temperature against the observations.
It is possible to introduce a Mean Bias in the model due to a mismatch of the
boundary forcing and the model interior.

$$ \text{MB} = \mathbf{\overline{m}} - \mathbf{\overline{o}}$$

In [16]:
from ioos_tools.ioos import stations_keys


def rename_cols(df, config):
    cols = stations_keys(config, key='station_name')
    return df.rename(columns=cols)

In [17]:
from ioos_tools.ioos import to_html, save_html, load_ncs
from ioos_tools.skill_score import mean_bias, apply_skill

dfs = load_ncs(config)

df = apply_skill(dfs, mean_bias, remove_mean=False, filter_tides=False)
skill_score = dict(mean_bias=df.to_dict())

# Filter out stations with no valid comparison.
df.dropna(how='all', axis=1, inplace=True)
df = df.applymap('{:.2f}'.format).replace('nan', '--')

# Pretty HTML table.
if not df.empty:
    df = rename_cols(df, config)
    html = to_html(df.T)
    fname = os.path.join(save_dir, 'mean_bias.html')
    save_html(fname, html)
    html

### Skill 2: Central Root Mean Squared Error

Root Mean Squared Error of the deviations from the mean.

$$ \text{CRMS} = \sqrt{\left(\mathbf{m'} - \mathbf{o'}\right)^2}$$

where: $\mathbf{m'} = \mathbf{m} - \mathbf{\overline{m}}$ and $\mathbf{o'} = \mathbf{o} - \mathbf{\overline{o}}$

In [18]:
from ioos_tools.skill_score import rmse

dfs = load_ncs(config)

df = apply_skill(dfs, rmse, remove_mean=True, filter_tides=False)
skill_score['rmse'] = df.to_dict()

# Filter out stations with no valid comparison.
df.dropna(how='all', axis=1, inplace=True)
df = df.applymap('{:.2f}'.format).replace('nan', '--')

# Pretty HTML table.
if not df.empty:
    df = rename_cols(df, config)
    html = to_html(df.T)
    fname = os.path.join(save_dir, 'rmse.html')
    save_html(fname, html)
    html

### Skill 3: R$^2$
https://en.wikipedia.org/wiki/Coefficient_of_determination

In [19]:
from ioos_tools.skill_score import r2

dfs = load_ncs(config)

df = apply_skill(dfs, r2, remove_mean=True, filter_tides=False)
skill_score['r2'] = df.to_dict()

# Filter out stations with no valid comparison.
df.dropna(how='all', axis=1, inplace=True)
df = df.applymap('{:.2f}'.format).replace('nan', '--')

# Pretty HTML table.
if not df.empty:
    df = rename_cols(df, config)
    html = to_html(df.T)
    fname = os.path.join(save_dir, 'r2.html')
    save_html(fname, html)
    html

In [20]:
import json

fname = os.path.join(save_dir, 'skill_score.json')

# Stringfy keys for json.
for key in skill_score.keys():
    skill_score[key] = {str(k): v for k, v in skill_score[key].items()}

with open(fname, 'w') as f:
    f.write(json.dumps(skill_score))

### Normalized Taylor diagrams

The radius is model standard deviation error divided  by observations deviation,
azimuth is arc-cosine of cross correlation (R), and distance to point (1, 0) on the
abscissa is Centered RMS.

In [21]:
%matplotlib inline
import matplotlib.pyplot as plt
from ioos_tools.taylor_diagram import TaylorDiagram


def make_taylor(samples):
    fig = plt.figure(figsize=(9, 9))
    dia = TaylorDiagram(samples['std']['OBS_DATA'],
                        fig=fig,
                        label="Observation")
    # Add samples to Taylor diagram.
    samples.drop('OBS_DATA', inplace=True)
    for model, row in samples.iterrows():
        dia.add_sample(row['std'], row['corr'], marker='s', ls='',
                       label=model)
    # Add RMS contours, and label them.
    contours = dia.add_contours(colors='0.5')
    plt.clabel(contours, inline=1, fontsize=10)
    # Add a figure legend.
    kw = dict(prop=dict(size='small'), loc='upper right')
    fig.legend(dia.samplePoints,
               [p.get_label() for p in dia.samplePoints],
               numpoints=1, **kw)
    return fig

```python
import numpy as np
import pandas as pd

dfs = load_ncs(config)

# Bin and interpolate all series.
freq = '30min'
for station, df in list(dfs.iteritems()):
    df = df.resample(freq).interpolate().dropna(axis=1)
    if 'OBS_DATA' in df:
        samples = pd.DataFrame.from_dict(dict(std=df.std(),
                                              corr=df.corr()['OBS_DATA']))
    else:
        continue
    samples[samples < 0] = np.NaN
    samples.dropna(inplace=True)
    if len(samples) <= 2:  # 1 obs 1 model.
        continue
    fig = make_taylor(samples)
    fig.savefig(os.path.join(save_dir, '{}.png'.format(station)))
    plt.close(fig)
```

### Load skill_score

In [22]:
import json

fname = os.path.join(config['run_name'], 'skill_score.json')
with open(fname, 'r') as f:
    skill_score = json.loads(f.read())

In [23]:
import pandas as pd

mean_bias = pd.DataFrame.from_dict(skill_score['mean_bias'])
mean_bias = mean_bias.applymap('{:.2f}'.format).replace('nan', '--')

skill_score = pd.DataFrame.from_dict(skill_score['rmse'])
skill_score = skill_score.applymap('{:.2f}'.format).replace('nan', '--')

In [24]:
from ioos_tools.ioos import make_map

bbox = config['region']['bbox']
units = config['units']
run_name = config['run_name']

kw = dict(zoom_start=11, line=True, states=False,
          secoora_stations=False, layers=False)
mapa = make_map(bbox, **kw)

### Clusters

In [25]:
all_obs = stations_keys(config)

In [26]:
from glob import glob
from operator import itemgetter

import iris
import folium
from folium.plugins import MarkerCluster

iris.FUTURE.netcdf_promote = True

big_list = []
for fname in glob(os.path.join(save_dir, "*.nc")):
    if 'OBS_DATA' in fname:
        continue
    cube = iris.load_cube(fname)
    model = fname.split('-')[-1].split('.')[0]
    lons = cube.coord(axis='X').points
    lats = cube.coord(axis='Y').points
    stations = cube.coord('station_code').points
    models = [model]*lons.size
    lista = zip(models, lons.tolist(), lats.tolist(), stations.tolist())
    big_list.extend(lista)

big_list.sort(key=itemgetter(3))
df = pd.DataFrame(big_list, columns=['name', 'lon', 'lat', 'station'])
df.set_index('station', drop=True, inplace=True)
groups = df.groupby(df.index)


locations, popups = [], []
for station, info in groups:
    sta_name = all_obs[station]
    for lat, lon, name in zip(info.lat, info.lon, info.name):
        locations.append([lat, lon])
        popups.append('[{}]: {}'.format(name, sta_name))

MarkerCluster(locations=locations, popups=popups).add_to(mapa)

### Model and observations plots

In [27]:
import warnings

# Suppresing warnings for a "pretty output."
# Remove this line to debug any possible issues.
warnings.simplefilter("ignore")

In [28]:
# Legend dictionary. If any new model is found we will use its filename as legend.
# Here we only provide some nice names for the models we expect to find.

titles = {
    'coawst_4_use_best': 'COAWST_4',
    'global': 'HYCOM',
    'NECOFS_GOM3_FORECAST': 'NECOFS_GOM3',
    'NECOFS_FVCOM_OCEAN_MASSBAY_FORECAST': 'NECOFS_MassBay',
    'OBS_DATA': 'Observations'
}

In [29]:
from bokeh.resources import CDN
from bokeh.plotting import figure
from bokeh.embed import file_html
from bokeh.models import HoverTool
from itertools import cycle
from bokeh.palettes import Spectral6

from folium.element import IFrame

# Plot defaults.
colors = Spectral6
colorcycler = cycle(colors)
tools = "pan,box_zoom,reset"
width, height = 750, 250


def make_plot(df, station):
    p = figure(toolbar_location="above",
               x_axis_type="datetime",
               width=width,
               height=height,
               tools=tools,
               title=str(station))
    for column, series in df.iteritems():
        series.dropna(inplace=True)
        if not series.empty:
            line = p.line(
                x=series.index,
                y=series.values,
                legend="%s" % titles.get(column, column),
                line_color=next(colorcycler),
                line_width=5,
                line_cap='round',
                line_join='round'
            )
            if 'OBS_DATA' not in column:
                bias = mean_bias[str(station)][column]
                skill = skill_score[str(station)][column]
            else:
                skill = bias = 'NA'
            p.add_tools(HoverTool(tooltips=[("Name", "%s" % column),
                                            ("Bias", bias),
                                            ("Skill", skill)],
                                  renderers=[line]))
    return p


def make_marker(p, station):
    lons = stations_keys(config, key='lon')
    lats = stations_keys(config, key='lat')

    lon, lat = lons[station], lats[station]
    html = file_html(p, CDN, station)
    iframe = IFrame(html, width=width+40, height=height+80)

    popup = folium.Popup(iframe, max_width=2650)
    icon = folium.Icon(color='green', icon='stats')
    marker = folium.Marker(location=[lat, lon],
                           popup=popup,
                           icon=icon)
    return marker

In [30]:
dfs = load_ncs(config)

for station in dfs:
    sta_name = all_obs[station]
    df = dfs[station]
    if df.empty:
        continue
    p = make_plot(df, station)
    maker = make_marker(p, station)
    maker.add_to(mapa)

In [31]:
mapa

In [32]:
elapsed = time.time() - start_time
log.info('{:.2f} minutes'.format(elapsed/60.))
log.info('EOF')

logfile = os.path.join(config['run_name'], 'log.txt')

with open(logfile) as f:
    print(f.read())

05:03:12 INFO: Saving data inside directory /home/filipe/IOOS/notebooks_demos/notebooks/latest
05:03:12 INFO: *********************** Run information ************************
05:03:12 INFO: Run date: 2016-11-21 20:03:12
05:03:12 INFO: Start: 2016-11-16 00:00:00
05:03:12 INFO: Stop: 2016-11-25 00:00:00
05:03:12 INFO: Bounding box: -71.30, 42.03,-70.57, 42.63
05:03:12 INFO: ********************* Catalog information **********************
05:03:12 INFO: ***************************** CSW ******************************
05:03:12 INFO: URL: http://www.ngdc.noaa.gov/geoportal/csw
05:03:14 INFO: CSW version: 2.0.2
05:03:14 INFO: Number of datasets available: 1
05:03:14 INFO: HYbrid Coordinate Ocean Model (HYCOM): Global
05:03:14 INFO: ***************************** DAP ******************************
05:03:14 INFO: http://oos.soest.hawaii.edu/thredds/dodsC/pacioos/hycom/global.html
05:03:14 INFO: ***************************** CSW ******************************
05:03:14 INFO: URL: https://dev-cata